<a href="https://colab.research.google.com/github/danh21/AI_investigation/blob/main/neural_network_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
#load datasheet
print ('Load MNIST Database')
mnist = tf.keras.datasets.mnist
(x_train,y_train), (x_test, y_test) = mnist.load_data()
x_train = np.reshape(x_train,(60000,784))/255.0
x_test = np.reshape(x_test,(10000,784))/255.0
y_train = np.matrix(np.eye(10)[y_train])
y_test = np.matrix(np.eye(10)[y_test])
print("------------------------------------------")
print(x_train.shape)
print(y_train.shape)

Load MNIST Database
11501568/11490434 [==============================] - 0s 0us/step
------------------------------------------
(60000, 784)
(60000, 10)


In [ ]:
def sigmoid(x):
  return 1./(1.+np.exp(-x))
def softmax(x):
  return np.divide(np.matrix(np.exp(x)),np.mat(np.sum(np.exp(x),axis=1)))
def ReLU(x):
  return np.maximum(x,0)
def Forwardpass(x,Wh1,bh1,Wh2,bh2,Wo,bo):
  zh1 = x@Wh1.T + bh1
  a1 = ReLU(zh1)
  zh2 = a1@Wh2.T + bh2
  a2 = sigmoid(zh2)
  z = a2@Wo.T + bo
  o = softmax(z)
  return o
def AccTest(label,prediction): #calculate the matching score
  OutMaxArg = np.argmax(prediction,axis=1)
  LabelMaxArg = np.argmax(label,axis=1)
  Accuracy = np.mean(OutMaxArg==LabelMaxArg)
  return Accuracy

In [ ]:
#define the network parameter
learningRate = 0.5
Epoch = 10
NumTrainSamples = 60000
NumTestSamples = 10000
NumInputs = 784
NumHiddenUnits = 512
# NumHiddenUnits1 = 512
NumClasses = 10

#initial weights
#hidden layer 1
Wh1 = np.matrix(np.random.uniform(-0.5,0.5,(NumHiddenUnits,NumInputs)))
bh1 = np.random.uniform(0,0.5,(1,NumHiddenUnits))
dWh1 = np.zeros((NumHiddenUnits,NumInputs))
dbh1 = np.zeros((1,NumHiddenUnits))
#hidden layer 2
Wh2 = np.matrix(np.random.uniform(-0.5,0.5,(NumHiddenUnits,NumHiddenUnits)))
bh2 = np.random.uniform(0,0.5,(1,NumHiddenUnits))
dWh2 = np.zeros((NumHiddenUnits,NumHiddenUnits))
dbh2 = np.zeros((1,NumHiddenUnits))
#Output layer
Wo = np.random.uniform(-0.5,0.5,(NumClasses,NumHiddenUnits))
bo = np.random.uniform(0,0.5,(1,NumClasses))
dWo = np.zeros((NumClasses,NumHiddenUnits))
dbo = np.zeros((1,NumClasses))

In [ ]:
#Test accuracy with random initial weights
prediction = Forwardpass(x_train,Wh1,bh1,Wh2,bh2,Wo,bo)
Acc = AccTest(y_train,prediction)
print(Acc)

0.129


In [ ]:
# from IPython.display import clear_output
# import matplotlib.pyplot as plt
# loss = []
# Acc = []
# for ep in range (Epoch):
#   #feed fordware propagation
#   x = x_train
#   y = y_train
#   zh1 = x@Wh1.T + bh1
#   a1 = ReLU(zh1)
#   zh2 = a1@Wh2.T + bh2
#   a2 = sigmoid(zh2)
#   z = a2@Wo.T + bo
#   o = softmax(z)
#   #calculate loss
#   loss.append(-np.sum(np.multiply(y,np.log10(o))))
#   #calculate the error for the output layer
#   d = o - y
#   #back propagate error
#   dh2 = d@Wo
#   dh2s = np.multiply(np.multiply(dh2,a2),(1-a2))
#   dh1 = dh2s@Wh2
#   #update weight
#   dWo = np.matmul(np.transpose(d),a2)
#   dbo = np.mean(d) #consider a is 1 for bias
#   dWh2 = np.matmul(np.transpose(dh2s),a1)
#   dbh2 = np.mean(dh2s) #consider a is 1 for bias
#   dWh1 = np.matmul(np.transpose(dh1),x)
#   dbh1 = np.mean(dh1) #consider a is 1 for bias
#   Wo = Wo - learningRate * dWo/NumTrainSamples
#   bo = bo - learningRate * dbo
#   Wh2 = Wh2 - learningRate * dWh2/NumTrainSamples
#   bh2 = bh2 - learningRate * dbh2
#   Wh1 = Wh1 - learningRate * dWh1/NumTrainSamples
#   bh1 = bh1 - learningRate * dbh1
#   #Test accuracy with random initial weights
#   prediction = Forwardpass(x_train,Wh1,bh1,Wh2,bh2,Wo,bo)
#   Acc.append(AccTest(y_test,prediction))
#   clear_output(wait=True)
#   plt.plot([i for i, _ in enumerate(Acc)],Acc,'o')
#   plt.show() #càng ngày càng chính xác hơn

In [ ]:
# #Test accuracy with random initial weights
# prediction = Forwardpass(x_train,Wh1,bh1,Wh2,bh2,Wo,bo)
# Acc = AccTest(y_train,prediction)
# print(Acc)

0.07296666666666667


In [ ]:
#mini-batch SGD
from IPython.display import clear_output
import matplotlib.pyplot as plt
loss = []
Acc = []
Batch_size = 200
Stochastic_samples = np.arange(NumTrainSamples) # 0,1,2,3,4,5,6,7,...,59999
for ep in range (Epoch):
  np.random.shuffle(Stochastic_samples)
  for ite in range (0,NumTrainSamples,Batch_size):
    #feed fordware propagation
    Batch_samples = Stochastic_samples[ite:ite+Batch_size]
    x = x_train[Batch_samples,:]
    y = y_train[Batch_samples,:]
    zh1 = x@Wh1.T + bh1
    a1 = ReLU(zh1)
    zh2 = a1@Wh2.T + bh2
    a2 = sigmoid(zh2)
    z = a2@Wo.T + bo
    o = softmax(z)
    #calculate loss
    loss.append(-np.sum(np.multiply(y,np.log10(o))))
    #calculate the error for the output layer
    d = o - y
    #back propagate error
    dh2 = d@Wo
    dhs2 = np.multiply(np.multiply(dh2,a2),(1-a2))
    dh1 = dhs2@Wh2
    #update weight
    dWo = np.matmul(np.transpose(d),a2)
    dbo = np.mean(d) #consider a is 1 for bias
    dWh2 = np.matmul(np.transpose(dhs2),a1)
    dbh2 = np.mean(dhs2) #consider a is 1 for bias
    dWh1 = np.matmul(np.transpose(dh1),x)
    dbh1 = np.mean(dh1) #consider a is 1 for bias
    Wo = Wo - learningRate * dWo/Batch_size
    bo = bo - learningRate * dbo
    Wh2 = Wh2 - learningRate * dWh2/Batch_size
    bh2 = bh2 - learningRate * dbh2
    Wh1 = Wh1 - learningRate * dWh1/Batch_size
    bh1 = bh1 - learningRate * dbh1
  #Test accuracy with random initial weights
  prediction = Forwardpass(x_test,Wh1,bh1,Wh2,bh2,Wo,bo)
  Acc.append(AccTest(y_test,prediction))
  print('Epoch:',ep)
  print('Accuracy:',AccTest(y_test,prediction))


Epoch: 0
Accuracy: 0.935
Epoch: 1
Accuracy: 0.945
Epoch: 2
Accuracy: 0.9489
Epoch: 3
Accuracy: 0.9565
Epoch: 4
Accuracy: 0.9584
Epoch: 5
Accuracy: 0.9575
Epoch: 6
Accuracy: 0.9601
Epoch: 7
Accuracy: 0.9612
Epoch: 8
Accuracy: 0.9628
Epoch: 9
Accuracy: 0.964
